## 因果论

### 主要内容
* 因果模型概念
* 因果关系学习
* 干预条件预测
* 观察条件预测（贝叶斯网络推理）
* 参考资料

### 1 因果模型概念

基本形式
* 方程式因果模型（Functional Causal Model）：$x_i=f_i(pa_i,u_i),i=1,...,n$，其中$u_i$表示未能测量到的因变量，$x_i$由$pa_i$和$u_i$唯一确定
* 因果图（Causal Graph）：由方程式因果模型对应作出，其中$u_i$不表示在因果图中

操作定义：干预（Intervention）
* 干预方程式因果模型中的$x_i$，置其值为$x_i'$，意味着把因果模型中方程左边为$x_i$的方程改为$x_i=x_i'$，其他方程保持不变
* 由以上定义可得，对应的因果图将改成去掉$x_i$与所有$pa_i$之间的箭头，并置$x_i$的值为$x_i'$

进一步假设
* 假设1：因果图为有向无圈图（directed acyclic graph，DAG）
* 假设2：$u_i$之间两两独立。这一假设蕴含一种特殊情况：没有未能测量到的的共因变量

因果贝叶斯网络（Causal Bayesian Network）
* 同时满足假设1和假设2的因果图将满足因果马尔科夫条件：$p(x_i|pa_i)=p(x_i|x_{i,nond})$，成为贝叶斯网络
* 贝叶斯网络为有向无圈图概率模型，其定义并不依赖于因果模型，以下三种定义等价：
    * 马尔科夫条件（常见定义）：$p(x_i|pa_i)=p(x_i|x_{i,nond})$，其中$pa_{i}$为$x_i$的父节点，$x_{i,nond}$为$x_i$的所有非后代节点（非后代节点不包括$x_i$）
    * 联合概率分解：$p(x_1,...,x_n)=\prod_{i}^np(x_i|pa_i)$，其中$pa_{i}$为$x_i$的父节点
    * 由d-分离（d-separation）标准可确定条件独立：$(X\perp\!\!\!\perp Y|Z)_G\Rightarrow(X\perp\!\!\!\perp Y|Z)_P$
* 由因果图建立的贝叶斯网络又叫做因果贝叶斯网络
* 贝叶斯网络的用途：利用贝叶斯网络进行观察预测时，联合概率分解可简化概率推理时的概率计算
* 因果贝叶斯网络的干预公式：$$p(x_1,...,x_n|\hat x_i')=\prod_{j\neq i}p(x_j|pa_j)$$

### 2 因果关系学习

目标
* 通过观察数据确定因果图结构

方法
* 在因果模型域中筛选出满足数据中条件独立的因果图（主要介绍这种方法）
* 利用数据更新不同因果图的先验概率（贝叶斯方法）

筛选条件
1. 因果马尔科夫条件：$p(x_i|pa_i)=p(x_i|x_{i,nond})$
    * 满足假设1，为有向无圈图
    * 满足假设2，$u_i$之间两两独立
2. 最小化条件（Minimality，奥卡姆剃刀）：当模型已经满足因果马尔科夫条件时，其任意子图不能满足因果马尔科夫条件
3. 稳定性条件（Stability/faithfulness）：$(X\perp\!\!\!\perp Y|Z)_P\Rightarrow(X\perp\!\!\!\perp Y|Z)_G$

特别说明

* 三个筛选条件并不独立，满足稳定性条件和因果马尔科夫条件的模型一定满足最小化条件，而满足最小化条件和因果马尔科夫条件的模型不一定满足稳定性条件。 为满足所有3个筛选条件，只需要同时满足稳定性条件和因果马尔科夫条件即可
* 由贝叶斯网络的定义，因果马尔科夫条件等价于：$(X\perp\!\!\!\perp Y|Z)_G\Rightarrow(X\perp\!\!\!\perp Y|Z)_P$，其又等价于：$(X\not\perp Y|Z)_P\Rightarrow(X\not\perp Y|Z)_G$。所以因果马尔科夫条件+稳定性条件等价于：$(X\perp Y|Z)_P=(X\perp Y|Z)_G$
* 若因果图有未能测量到的的共因变量时，其将不再满足因果马尔科夫条件。若要对这类因果模型进行筛选，可将未测量到的共因变量在图中显式地表示出来，记做潜在变量（latent variables）。这一含有潜在变量的因果图将满足因果马尔科夫条件，因此可应用上述筛选条件进行筛选

#### 因果图不含潜在变量筛选算法
筛选结果
* 模式图（pattern）/d-分离标准等价图
* 连接方式解读
    * $ a\rightarrow b$：a为b的直接原因
    * $ a-b$：a为b的直接原因，或b为a的直接原因

算法示例
* IC算法（Inductive Causation）
    1. 从没有连接的图开始，若$(a\perp\!\!\!\perp b|S_{ab})_P$对所有$S_{ab}$都不成立，则增加a、b之间的无向连接
    2. 对撞结构测试：$a-c-b$，若$c\notin S_{ab}$，则$a\rightarrow c\leftarrow b$
    3. 利用以下原则进一步确定有向连接的方向：
        * 不产生新的对撞结构
        * 不产生有向循环
* SGS算法
    * 与IC算法相似，只是完成第一步的方向不同：从全连接图开始，若$(a\perp\!\!\!\perp b|S_{ab})_P$对某个$S_{ab}$成立，则去掉a、b之间的无向连接
* PC算法
    * 对SGS算法的第一步进行简化，判定$(a\perp\!\!\!\perp b|S_{ab})_P$时：
        * $S_{ab}$的个数从0到多
        * $S_{ab}$限定为与a相邻的点组成集合的子集

#### 因果图含潜在变量筛选算法
筛选结果
* Partially Oriented Inducing Path Graph/带标号的模式图（marked pattern）
* 连接方式解读
    * $ a\mathop \rightarrow \limits^\ast b$：a为b的真实原因，a、b之间不存在未测量到的共因
    * $ a\leftrightarrow b$：a、b之间存在未测量到的共因（同时a、b之间可能存在因果关系）
    * $ a\rightarrow b$：a为b的潜在原因，即要么a为b的真实原因，要么a、b之间存在未测量到的共因
    * $ a-b$：a为b的潜在原因，或b为a的潜在原因，或存在未测量到的共因

算法示例
* IC* 算法
    1. 从没有连接的图开始，若$(a\perp\!\!\!\perp b|S_{ab})_P$对所有$S_{ab}$都不成立，则增加a、b之间的无向连接
    2. 对撞结构测试：$a-c-b$，若$c\notin S_{ab}$，则$a\rightarrow c\leftarrow b$
    3. 利用以下条件进一步确定有向连接的方向：
        * 远离对撞结构测试：$a\rightarrow c-b$，若$c\notin S_{ab}$，则$a\rightarrow c\leftarrow b$；若$c\in S_{ab}$，则$a\rightarrow c\mathop \rightarrow \limits^\ast b$
        * 不产生有向循环：若$a-b$，且存在另外一条从a到b的严格有向路径（节点间均为真实原因关系，不存在共因），那么把$a-b$改成$a\rightarrow b$
* CI（Casual Inference）算法
    1. 从全连接图开始，若$(a\perp\!\!\!\perp b|S_{ab})_P$对某个$S_{ab}$成立，则去掉a、b之间的无向连接
    2. 对撞结构测试：$a-c-b$，若$c\notin S_{ab}$，则$a\rightarrow c\leftarrow b$
    3. 利用以下条件进一步确定有向连接的方向：
        * 不产生有向循环：若$a-b$，且存在另外一条从a到b的严格有向路径（节点间均为真实原因关系，不存在共因），那么把$a-b$改成$a\rightarrow b$
        * 若$a \rightarrow c\leftarrow b$，$c-d$，而$d\in S_{ab}$，则把c、d之间的连接改为$c\leftarrow d$
        * 若U是a到b的关于m的明确辨别路径（definite discriminating path），p和r在路径U上与m相邻，且$p-m-r$形成三角形结构。若$m\not\in S_{ab}$，则：$p\rightarrow m\leftarrow r$。U是a到b的关于m的明确辨别路径需满足：
            * U为a到b的非有向路径，$m\neq a$，$m\neq b$，除m、a和b外，U上的节点要么为对撞结构，要么为绝对非对撞结构：$a\mathop \leftarrow \limits^\ast b-c$，$a-b\mathop \rightarrow \limits^\ast c$
            * 若路径U上V和V'相邻，且V'在m和V之间，那么$V\rightarrow V'$
            * 若路径U上V在a和m之间，且V是对撞结构，那么$V\mathop \rightarrow \limits^\ast b$，或$V\leftarrow b$
            * 若路径U上V在b和m之间，且V是对撞结构，那么$V\mathop \rightarrow \limits^\ast a$，或$V\leftarrow a$
            * a、b不相邻
        * 远离对撞结构测试：$a\rightarrow c-b$，若$c\notin S_{ab}$，则$a\rightarrow c\leftarrow b$；若$c\in S_{ab}$，则$a\rightarrow c\mathop \rightarrow \limits^\ast b$

* FCI（Fast Causal Inference）算法
    1. 从全连接图开始，若$(a\perp\!\!\!\perp b|S_{ab})_P$对某个$S_{ab}$成立，则去掉a、b之间的无向连接。判定$(a\perp\!\!\!\perp b|S_{ab})_P$时：
        * $S_{ab}$的个数从0到多
        * $S_{ab}$限定为与a相邻的点组成集合的子集
    2. 对撞结构测试：$a-c-b$，若$c\notin S_{ab}$，则$a\rightarrow c\leftarrow b$
    3. 若$(a\perp\!\!\!\perp b|S_{ab})_P$对$S_{ab}$成立，则去掉a、b之间的无向连接。$S_{ab}$为Possible-D-SEP(a,b)的某个子集，Possible-D-SEP(a,b)是排除了一定不在D-Sep(a,b)集合中的点组成的集合，V$\in$D-Sep(a,b)当且仅当：a与V之间存在一条非有向路径U，且U上除a、b外每个节点都是a、b的祖先节点，并且为对撞结构
    4. 将有向连接改为无向连接，按照CI算法的第2、3步重新确定连接方向
    
    
    
#### X为Y的真实原因判定条件
* 不带时间信息
    * X与Y在所有条件下都相关
    * 存在变量Z是X的潜在原因
    * $Z\not\perp  Y|S$
    * $Z\perp\!\!\!\perp  Y|S\cup X$
* 带时间信息
    * Z和S都在X之前发生
    * $Z\not\perp  Y|S$
    * $Z\perp\!\!\!\perp  Y|S\cup X$

### 3 干预结果预测

目标
* 通过观察数据和基于因果假设建立的因果贝叶斯网络预测干预结果

#### 因果贝叶斯网络的干预公式
$$p(x_1,...,x_n|\hat x_i')=\prod_{j\neq i}p(x_j|pa_j)$$
    
#### 直接原因调整公式

$$P(y|\hat x_i')=\sum_{pa_i}P(y|x_i',pa_i)P(pa_i)$$

#### 后门调整公式
$$P(y|\hat x)=\sum_zP(y|x,z)P(z)$$

其中变量组$Z$满足后门标准：
* $Z$中没有$X$的后代节点
* $Z$阻断了所有从$X$到$Y$且含有指向$X$箭头的路径

#### 前门调整公式
$$P(y|\hat x)=\sum_zP(z|x)\sum_{x'}P(y|x',z)P(x')$$

其中变量组$Z$满足前门标准：
* $Z$阻断了所有从$X$到$Y$的直接路径
* 从$X$到$Z$没有未被阻断的后门路径
* 所有从$Z$到$Y$的后门路径都被$X$阻断

#### 干预运算规则
* 规则1（添加/删除观察条件）：$(Y\perp Z|X,W)_{G_\overline X}\Rightarrow P(y|\hat x,z,w)=P(y|\hat x,w)$，
* 规则2（交换干预/观察条件）： $(Y\perp Z|X,W)_{G_{\overline X \underline Z}}\Rightarrow P(y|\hat x,\hat z,w)=P(y|\hat x,z,w)$
* 规则3（添加/删除干预条件）： $(Y\perp Z|X,W)_{G_{\bar X \overline {Z(W)}}}\Rightarrow P(y|\hat x,\hat z,w)=P(y|\hat x,w)$，其中 Z(W)为节点组Z的子集，且不能是W的祖先节点

（记）当条件变量到预测变量的：
* 前后门路径均被阻断时，可添加/删除观察条件
* 后门路径被阻断时，可把干预条件换成观察条件，或把观察条件换成干预条件
* 前门路径无条件阻断，或前后门路径均被阻断时，可添加/删除干预条件

### 4 观察条件预测（贝叶斯网络推理）

#### 构建模型

连续型变量进行概率更新
* 若需精确解，需满足：
    * 连续型变量节点的条件概率密度为正态分布。对每一组离散型父节点状态组合，方差$\sigma_c^2$为常数，期望$\mu_c$为连续型父节点的线性组合。
    * 离散型变量不能有连续型父节点
* 若允许近似解，则可对连续型变量进行离散处理转化成离散型变量

#### 信念更新

目标

* 计算证据更新后变量$A$的概率分布：$P(A|e)=\frac{P(A,e)}{\sum_{A}P(A,e)}$，其中$P(A,e)=\sum_{U-A}\prod_{i=1}^n P(A_i|pa(A_i))\prod_{j=1}^m e_j$

运算法则

* 条件概率看做表，两个表相乘得到新的表。
* 乘以证据看做只保留表中符合证据的数据，其他数据置为0。

难点

* 需要有效的算法计算连乘求和：$P(A,e)=\sum_{U-A}\prod_{i=1}^n P(A_i|pa(A_i))\prod_{j=1}^m e_j$

**联合树（Junction Tree）算法**
* 联合树算法基于变量消元法（variable elimination）：使用分配律降低连乘求和计算量
* 需要解决的问题：确定最佳的变量消元顺序（对变量进行连乘求和的顺序）
* 解决步骤：
    1. 由贝叶斯网络生成域图（domain graph）
        * 域图：与贝叶斯网络结构相同，有向连接变成无向连接，有共同子节点的父节点两两之间增加无向连接
    2. 若域图为可三角化图（triangulated graph），由域图的团（clique）组织成连接树（join tree）。
    
        概念解释：
        * 消元（elimination）：在域图中移除节点，去掉与之相连的连接线。与移除节点相连的节点两两之间若本没有连接，增加无向连接
        * 完美消元（perfect elimination）：没有增加新连接的消元
        * 可三角化图：存在完美消元序列（perfect elimination sequences）的无向图
        * 完全集：集合中所有的节点两两相连
        * 团：由域图节点组成的极大完全集（maximal complete set，不是其他完全集的子集）
        * 连接树：树节点由域图的团组成，树中任意两个节点之间路径上的所有节点包含这两个树节点的交集
    3. 从连接树中依次消除单节点（simplicial nodes），形成完美消元序列
        * 单节点：相连节点组成完全集的节点。
    4. 任意一个以变量$A$结尾的完美消元序列都是计算$P(A)$的最佳消元序列
    5. 在联合树中进行信念传递（message passing）
        * 联合树：连接树+势函数（条件概率表）

其他说明
* 若域图不能被三角化，将其嵌入三角化域图，然后再使用联合树算法
* 随机模拟方法
    * 吉布斯采样

### 参考资料

[1] Pearl, J. (2009). Causality. Cambridge: Cambridge University Press. doi:10.1017/CBO9780511803161

[2] Spirtes et al., 1993 P. Spirtes, C.N. Glymour, and R. Scheines. Causation, Prediction, and Search. Springer-Verlag, New York, 1993.

[3] Finn V. Jensen and Thomas D. Nielsen. 2007. Bayesian Networks and Decision Graphs (2nd. ed.). Springer Publishing Company, Incorporated.